In [7]:
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
BASE_DIR = Path("..")
DATA_DIR = BASE_DIR / "data"

VIDEO_META_PATH = DATA_DIR / "channel_videos_with_stats.csv"

df_videos_full = pd.read_csv(VIDEO_META_PATH)

df_videos_full.shape

(5157, 10)

In [4]:
# 채널별 Top-N LONG 영상 선정
TOP_N = 3

top_videos = (
    df_videos_full
    .query("video_type == 'LONG'")
    .sort_values("view_count", ascending=False)
    .groupby("channel_label")
    .head(TOP_N)
    .reset_index(drop=True)
)

top_videos[[
    "channel_label",
    "video_title",
    "view_count",
    "video_type"
]]


,channel_label,video_title,view_count,video_type
0,IVE,IVE 아이브 'LOVE DIVE' DANCE PRACTICE,38833072,LONG
1,IVE,LOVE DIVE,27762844,LONG
2,IVE,GOLDEN Covered by IVE ANYUJIN,21459593,LONG
3,PLAVE,PLAVE(플레이브) 'Pump Up The Volume!' M/V,14928088,LONG
4,PLAVE,PLAVE(플레이브) 'WAY 4 LUV' M/V,14651364,LONG
5,PLAVE,PLAVE(플레이브) '여섯 번째 여름 (The 6th Summer)' M/V,14393882,LONG
6,RIIZE,RIIZE 라이즈 'Love 119' Performance Video,8256579,LONG
7,RIIZE,RIIZE 라이즈 '잉걸 (Ember to Solar)' Track Video,6964322,LONG
8,RIIZE,RIIZE 라이즈 'Siren' Dance Practice,6828183,LONG
9,ISEGYE_IDOL,이세계아이돌(ISEGYE IDOL) 'Nameless' Official Perfor...,3957673,LONG


In [8]:
from comments import fetch_all_comments

all_comments = []

for i, row in top_videos.iterrows():
    video_id = row["video_id"]
    title = row["video_title"]
    channel = row["channel_label"]

    print(f"\n[{i+1}/{len(top_videos)}] {channel} | {title}")

    comments = fetch_all_comments(video_id)
    print(f"→ collected {len(comments)} comments")

    for c in comments:
        c["channel_label"] = channel
        c["video_title"] = title

    all_comments.extend(comments)



[1/12] IVE | IVE 아이브 'LOVE DIVE' DANCE PRACTICE
[comments] video=Bo2aD_I7-1U | page 1 | total=100
[comments] video=Bo2aD_I7-1U | page 2 | total=200
[comments] video=Bo2aD_I7-1U | page 3 | total=300
[comments] video=Bo2aD_I7-1U | page 4 | total=400
[comments] video=Bo2aD_I7-1U | page 5 | total=500
[comments] video=Bo2aD_I7-1U | page 6 | total=600
[comments] video=Bo2aD_I7-1U | page 7 | total=700
[comments] video=Bo2aD_I7-1U | page 8 | total=800
[comments] video=Bo2aD_I7-1U | page 9 | total=900
[comments] video=Bo2aD_I7-1U | page 10 | total=1000
[comments] video=Bo2aD_I7-1U | page 11 | total=1100
[comments] video=Bo2aD_I7-1U | page 12 | total=1200
[comments] video=Bo2aD_I7-1U | page 13 | total=1300
[comments] video=Bo2aD_I7-1U | page 14 | total=1400
[comments] video=Bo2aD_I7-1U | page 15 | total=1500
[comments] video=Bo2aD_I7-1U | page 16 | total=1600
[comments] video=Bo2aD_I7-1U | page 17 | total=1700
[comments] video=Bo2aD_I7-1U | page 18 | total=1800
[comments] video=Bo2aD_I7-1U | pa